# BATTLE OF THE NEIGHBORHOODS

### Import The Libraries

In [5]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import Image to display JPEG files
from IPython.display import Image

print('Libraries imported.')

Libraries imported.


### Get The Data For the Three Capitals

In [2]:
mexicocity_neighborhoods = pd.read_csv('MexicoCity.csv')
mexicocity_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Location,Latitude,Longitude
0,1020,Álvaro Obregón,San José Insurgentes,NaN,19.366474,-99.185502
1,1110,Álvaro Obregón,Las Palmas,NaN,19.399131,-99.211264
2,1790,Álvaro Obregón,Lomas de San Ángel Inn,NaN,19.344786,-99.206145
3,2460,Azcapotzalco,La Preciosa,NaN,19.486607,-99.194068
4,2630,Azcapotzalco,Coltongo,NaN,19.482953,-99.152960


In [3]:
ottawa_neighborhoods = pd.read_csv('Ottawa.csv')
ottawa_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Location,Latitude,Longitude
0,K1A,West Block,Parliament Hill,K1A West Block,45.423594,-75.700929
1,K1A 0A1,K1A 0A1,K1A 0A1,K1A 0A1,45.445473,-75.689448
2,K1A 0A2,K1A 0A2,K1A 0A2,K1A 0A2,45.444840,-75.691540
3,K1A 0A3,K1A 0A3,K1A 0A3,K1A 0A3,45.423382,-75.697892
4,K1A 0A4,K1A 0A4,K1A 0A4,K1A 0A4,45.423787,-75.697460


In [4]:
washingtondc_neighborhoods = pd.read_csv('WashingtonDC.csv')
washingtondc_neighborhoods.head()

,PostalCode,Borough,Neighborhood,Location,Latitude,Longitude
0,20020,Hillcrest,Hillcrest,20020,29.392196,-95.224491
1,20012,Shepherd Park,Shepherd Park,20012,29.817628,-95.409891
2,20019,River Terrace,River Terrace,20019,30.101878,-95.133057
3,20016,Spring Valley,Spring Valley,20016,32.729640,-116.966862
4,20007,Georgetown,Georgetown,20007,33.376834,-79.294496


# MEXICO CITY

### Explore Mexico City

In [3]:
# Get the geographical coordinates of Mexico City.
address = 'Mexico City'
geolocator = Nominatim(user_agent="mex_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mexico City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mexico City are 19.4326009, -99.1333416.


In [9]:
# create map of mexicocity using latitude and longitude value
map_mexicocity = folium.Map(location=[latitude, longitude], zoom_start=11)

# Display the map
map_mexicocity

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
# Create a function to repeat the same process to all the neighborhoods in Central Toronto
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get Foursquare Credentials

In [6]:
CLIENT_ID = 'XAJLSH3R3GW5PNAARSDELOM4E4OIZTV25NJAEZ4RTSQADLBT' # your Foursquare ID
CLIENT_SECRET = 'QCM5NXJAO1RWDDMVMSNIM0E5AEHNDMJJRWTFWLAK022YKKJU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XAJLSH3R3GW5PNAARSDELOM4E4OIZTV25NJAEZ4RTSQADLBT
CLIENT_SECRET:QCM5NXJAO1RWDDMVMSNIM0E5AEHNDMJJRWTFWLAK022YKKJU


### Explore Mexico City

In [11]:
# Get Mexico City Venues
LIMIT = 100
neighborhood_venues = getNearbyVenues(names=mexicocity_neighborhoods['Neighborhood'],
                                    latitudes=mexicocity_neighborhoods['Latitude'],
                                    longitudes=mexicocity_neighborhoods['Longitude']
                                    )

print(neighborhood_venues.shape)

San José Insurgentes
 Las Palmas
 Lomas de San Ángel Inn
 La Preciosa
 Coltongo
 Santa Cruz Acayucan
 Nueva Santa María
 San Salvador Xochimanca
 Narvarte
 Del Valle
 Alamos
 Moderna
 Nonoalco
 Nápoles
 San José Insurgentes
 Crédito Constructor
 del Carmen
 San Pablo Tepetlapa
 Santa Úrsula Coapa
 Lomas de Vista Hermosa
 Bosques de las Lomas
 Centro
 Juárez
 Roma
 Doctores
 Obrera
 Nonoalco Tlatelolco
 Héroe de Nacozari
 Industrial
 Tres Estrellas
 San Juan de Aragón
 Pantitlán
 Viaducto Piedad
 Santa Anita
 Granjas México
 Agrícola Oriental
 San Bernabé Ocotepec
 Las Palmas
 Lomas de Chapultepec
 Argentina Poniente
 Pensil
 Irrigación
 Los Morales Polanco
 Granada
 Palmitas Polanco
 Polanco
 Anzures
 Reforma Social
 Bosques de las Lomas
 Escandón
 Amóérica
San Miguel Chapultepec
 del Mar
 Tlalpan
 La Joya
 Romero Rubio
 Merced Centro
 Lorenzo Boturini
 Magdalena Mixhuca
 Jardín Balbuena
 San Bernardino
(2410, 7)


In [12]:
neighborhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,San José Insurgentes,19.366474,-99.185502,La Re Pizza,19.367800,-99.185008,Pizza Place
1,San José Insurgentes,19.366474,-99.185502,Central Onceveintitres,19.365681,-99.183898,Antique Shop
2,San José Insurgentes,19.366474,-99.185502,Parque de los Insurgentes,19.366153,-99.183370,Park
3,San José Insurgentes,19.366474,-99.185502,Jardin Interior,19.364073,-99.185962,Vegetarian / Vegan Restaurant
4,San José Insurgentes,19.366474,-99.185502,"Carnitas Y Ensaladas ""Pebra""",19.367374,-99.187515,Buffet


In [13]:
neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agrícola Oriental,35,35,35,35,35,35
Amóérica,2,2,2,2,2,2
Anzures,81,81,81,81,81,81
Argentina Poniente,4,4,4,4,4,4
Bosques de las Lomas,86,86,86,86,86,86
Centro,57,57,57,57,57,57
Coltongo,24,24,24,24,24,24
Crédito Constructor,84,84,84,84,84,84
Del Valle,97,97,97,97,97,97


In [14]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 249 uniques categories.


### Analyze Each Neighborhood In Mexico City

In [15]:
# Analyze Each Neighborhood
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cave,Cemetery,Chinese Restaurant,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distillery,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Event Space,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Radio Station,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sausage Shop,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Theater,Theme Restaurant,Track,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Warehouse Store,Watch Shop,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,San José Insurgentes,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [16]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped

,Neighborhood,Yoga Studio,Accessories Store,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bistro,Boarding House,Bookstore,Boutique,Bowling Alley,Bowling Green,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Buffet,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Campground,Candy Store,Cave,Cemetery,Chinese Restaurant,Circus,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Distillery,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Event Space,Fair,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Flea Market,Fondue Restaurant,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Theater,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Great Outdoors,Paella Restaurant,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Radio Station,Record Shop,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Road,Rock Climbing Spot,Rock Club,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Sausage Shop,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Squash Court,Stables,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Theater,Theme Restaurant,Track,Turkish Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Veterinarian,Video Game Store,Warehouse Store,Watch Shop,Water Park,Wine Bar,Wine Shop,Winery,Wings Joint
0,Agrícola Oriental,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.028571,0.000000,0.057143,0.000000,0.000000,0.028571,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.057143,0.000000,0.000000,0.000000,0.000000,0.028571,0.000000,0.0000,0.000000,0.00,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,0

In [17]:
neighborhood_grouped.shape

(58, 249)

In [18]:
# write a function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
#Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [20]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agrícola Oriental,Taco Place,Mexican Restaurant,Restaurant,Burger Joint,Bakery,Convenience Store,BBQ Joint,Pizza Place,Seafood Restaurant,Café
1,Amóérica,Bakery,Italian Restaurant,Empanada Restaurant,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flea Market,Fish & Chips Shop
2,Anzures,Mexican Restaurant,Coffee Shop,Café,Japanese Restaurant,Restaurant,Liquor Store,Health & Beauty Service,Fried Chicken Joint,Bar,Gym / Fitness Center
3,Argentina Poniente,Moving Target,Plaza,Skate Park,Fair,Electronics Store,Food & Drink Shop,Food,Fondue Restaurant,Flea Market,Fish & Chips Shop
4,Bosques de las Lomas,Ice Cream Shop,Sushi Restaurant,Restaurant,Coffee Shop,Breakfast Spot,Electronics Store,Bakery,Tea Room,Salad Place,Furniture / Home Store


In [21]:
# Cluster Neighborhoods
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 0, 4, 4, 4, 3, 4, 4, 4, 3], dtype=int32)

In [22]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

mexicocity_merged = mexicocity_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
mexicocity_merged = mexicocity_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

mexicocity_merged.head() # check the last columns!

,Unnamed: 0,PostalCode,Borough,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,1020,Álvaro Obregón,San José Insurgentes,NaN,19.366474,-99.185502,4.0,Mexican Restaurant,Coffee Shop,Ice Cream Shop,Bakery,Taco Place,Sandwich Place,Pizza Place,Farmers Market,Italian Restaurant,Seafood Restaurant
1,1,1110,Álvaro Obregón,Las Palmas,NaN,19.399131,-99.211264,4.0,Taco Place,Convenience Store,Pizza Place,Mexican Restaurant,Cafeteria,Brewery,Stables,Bistro,Sporting Goods Shop,Breakfast Spot
2,2,1790,Álvaro Obregón,Lomas de San Ángel Inn,NaN,19.344786,-99.206145,1.0,Taco Place,Café,Restaurant,Garden Center,General Entertainment,Pizza Place,Sushi Restaurant,Swim School,Breakfast Spot,Music Venue
3,3,2460,Azcapotzalco,La Preciosa,NaN,19.486607,-99.194068,3.0,Mexican Restaurant,Lounge,Bakery,Market,BBQ Joint,Gym,Ice Cream Shop,Convenience Store,Burrito Place,Burger Joint
4,4,2630,Azcapotzalco,Coltongo,NaN,19.482953,-99.152960,4.0,Mexican Restaurant,Clothing Store,Fast Food Restaurant,Ice Cream Shop,BBQ Joint,Coffee Shop,Sandwich Place,Snack Place,Breakfast Spot,Food Truck


In [23]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mexicocity_merged['Latitude'], mexicocity_merged['Longitude'], mexicocity_merged['Neighborhood'], mexicocity_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#        color=rainbow[cluster-1],
        fill=True,
#        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Mexico City Clusters Below

In [25]:
mexicocity_merged.loc[mexicocity_merged['Cluster Labels'] == 0, mexicocity_merged.columns[[1] + list(range(5, mexicocity_merged.shape[1]))]]

,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,11820,64.77054,-147.268434,0.0,Bakery,Italian Restaurant,Empanada Restaurant,Food Stand,Food Court,Food & Drink Shop,Food,Fondue Restaurant,Flea Market,Fish & Chips Shop


In [26]:
mexicocity_merged.loc[mexicocity_merged['Cluster Labels'] == 1, mexicocity_merged.columns[[1] + list(range(5, mexicocity_merged.shape[1]))]]

,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,1790,19.344786,-99.206145,1.0,Taco Place,Café,Restaurant,Garden Center,General Entertainment,Pizza Place,Sushi Restaurant,Swim School,Breakfast Spot,Music Venue
25,6800,19.415073,-99.139208,1.0,Taco Place,Mexican Restaurant,Bar,Bakery,Wings Joint,BBQ Joint,Indie Theater,Gym,Liquor Store,Fried Chicken Joint
28,7800,19.474962,-99.125778,1.0,Taco Place,Mexican Restaurant,Restaurant,Wings Joint,Park,Bakery,Snack Place,Ice Cream Shop,Veterinarian,Sushi Restaurant
34,8400,19.402445,-99.105140,1.0,Taco Place,Restaurant,Seafood Restaurant,Dessert Shop,Convenience Store,Furniture / Home Store,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Empanada Restaurant
35,8500,19.395608,-99.070934,1.0,Taco Place,Mexican Restaurant,Restaurant,Burger Joint,Bakery,Convenience Store,BBQ Joint,Pizza Place,Seafood Restaurant,Café
49,11800,19.401632,-99.178989,1.0,Taco Place,Mexican Restaurant,Bakery,Burger Joint,Bar,Coffee Shop,Restaurant,Ice Cream Shop,Park,Diner
57,15820,19.415902,-99.124786,1.0,Taco Place,Seafood Restaurant,Mexican Restaurant,Martial Arts Dojo,Park,Restaurant,Convenience Store,Snack Place,Farmers Market,Fast Food Restaurant


In [27]:
mexicocity_merged.loc[mexicocity_merged['Cluster Labels'] == 2, mexicocity_merged.columns[[1] + list(range(5, mexicocity_merged.shape[1]))]]

,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,6700,28.404122,-106.85044,2.0,Gym,Gym / Fitness Center,Steakhouse,Wings Joint,Farmers Market,English Restaurant,Event Space,Fair,Falafel Restaurant,Fast Food Restaurant


In [28]:
mexicocity_merged.loc[mexicocity_merged['Cluster Labels'] == 3, mexicocity_merged.columns[[1] + list(range(5, mexicocity_merged.shape[1]))]]

,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,2460,19.486607,-99.194068,3.0,Mexican Restaurant,Lounge,Bakery,Market,BBQ Joint,Gym,Ice Cream Shop,Convenience Store,Burrito Place,Burger Joint
5,2770,19.475752,-99.194579,3.0,Mexican Restaurant,Taco Place,Ice Cream Shop,BBQ Joint,Market,Diner,Cemetery,Café,Seafood Restaurant,Burger Joint
11,3510,19.395318,-99.134495,3.0,Mexican Restaurant,Taco Place,Bar,Gym,Seafood Restaurant,Bakery,Restaurant,Convenience Store,Pet Store,Burger Joint
18,4910,19.306661,-99.145487,3.0,Mexican Restaurant,Soccer Stadium,Light Rail Station,Motel,Sculpture Garden,Campground,Taco Place,Brewery,Creperie,Hotel
21,6400,18.921928,-99.230001,3.0,Mexican Restaurant,Ice Cream Shop,Restaurant,Coffee Shop,Park,Café,Nightclub,Hotel,Gluten-free Restaurant,Sandwich Place
24,6720,19.416965,-99.148986,3.0,Mexican Restaurant,Taco Place,Bar,Hotel,Coffee Shop,Flea Market,Burger Joint,Shopping Mall,Fried Chicken Joint,Spanish Restaurant
32,8200,19.401302,-99.131585,3.0,Mexican Restaurant,Bakery,Taco Place,Juice Bar,Café,Burger Joint,Fast Food Restaurant,Seafood Restaurant,Ice Cream Shop,Bar
33,8300,19.401020,-99.123317,3.0,Mexican Restaurant,Taco Place,Restaurant,Speakeasy,Soccer Stadium,Motorcycle Shop,Motel,Metro Station,Liquor Store,Gym
40,11400,19.450595,-99.201378,3.0,Mexican Restaurant,Taco Place,Restaurant,Bakery,Sandwich Place,Theater,Burger Joint,Pharmacy,Street Food Gathering,Brewery
54,14090,19.281359,-99.165162,3.0,Mexican Restaurant,Taco Place,Pharmacy,Burger Joint,Sports Club,Bakery,Sporting Goods Shop,Diner,Steakhouse,Coffee Shop


In [29]:
mexicocity_merged.loc[mexicocity_merged['Cluster Labels'] == 4, mexicocity_merged.columns[[1] + list(range(5, mexicocity_merged.shape[1]))]]

,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1020,19.366474,-99.185502,4.0,Mexican Restaurant,Coffee Shop,Ice Cream Shop,Bakery,Taco Place,Sandwich Place,Pizza Place,Farmers Market,Italian Restaurant,Seafood Restaurant
1,1110,19.399131,-99.211264,4.0,Taco Place,Convenience Store,Pizza Place,Mexican Restaurant,Cafeteria,Brewery,Stables,Bistro,Sporting Goods Shop,Breakfast Spot
4,2630,19.482953,-99.152960,4.0,Mexican Restaurant,Clothing Store,Fast Food Restaurant,Ice Cream Shop,BBQ Joint,Coffee Shop,Sandwich Place,Snack Place,Breakfast Spot,Food Truck
6,2800,19.463835,-99.168220,4.0,Mexican Restaurant,Ice Cream Shop,Taco Place,Café,Bakery,Coffee Shop,Sushi Restaurant,Restaurant,Burger Joint,Snack Place
7,2870,19.459635,-99.173887,4.0,Gastropub,Food Truck,Diner,Bar,Tennis Court,Mediterranean Restaurant,Arts & Crafts Store,Restaurant,Fish & Chips Shop,Flea Market
8,3020,19.401374,-99.147176,4.0,Seafood Restaurant,Mexican Restaurant,Taco Place,Burger Joint,Café,Bar,Vegetarian / Vegan Restaurant,Beer Bar,Pizza Place,Nail Salon
9,3100,19.394237,-99.167034,4.0,Mexican Restaurant,Seafood Restaurant,Coffee Shop,Bakery,Restaurant,Café,Taco Place,BBQ Joint,Pizza Place,Spa
12,3700,19.381314,-99.188323,4.0,Mexican Restaurant,Taco Place,Restaurant,Coffee Shop,Café,Food Truck,Bar,Convenience Store,Pet Store,Dance Studio
13,3810,19.393770,-99.176602,4.0,Coffee Shop,Mexican Restaurant,Café,Taco Place,Ice Cream Shop,Restaurant,Spa,Italian Restaurant,Japanese Restaurant,Pizza Place
14,3900,19.366474,-99.185502,4.0,Mexican Restaurant,Coffee Shop,Ice Cream Shop,Bakery,Taco Place,Sandwich Place,Pizza Place,Farmers Market,Italian Restaurant,Seafood Restaurant


# WASHINGTON DC

### Explore Washington DC

In [72]:
# Get the geographical coordinates of Washington DC.
address = 'Washington DC'
geolocator = Nominatim(user_agent="usa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Washington DC are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Washington DC are 38.8950092, -77.0365625.


In [73]:
# create map of washington using latitude and longitude value
map_washington = folium.Map(location=[latitude, longitude], zoom_start=11)

# Display the map
map_washington

In [32]:
# Get Washington DC Venues
LIMIT = 100
neighborhood_venues = getNearbyVenues(names=washingtondc_neighborhoods['Neighborhood'],
                                    latitudes=washingtondc_neighborhoods['Latitude'],
                                    longitudes=washingtondc_neighborhoods['Longitude']
                                    )

print(neighborhood_venues.shape)

Hillcrest
Shepherd Park
River Terrace
Spring Valley
Georgetown
Fort McNair
Fort Stanton
Hillandale
Hillbrook
Colonial Village
Forest Hills
Chevy Chase
Crestwood
Congress Heights
Douglass
Garfield Heights
Naylor Gardens
Barry Farm
Historic Anacostia
Fairfax Village
Buzzard Point
Fort Davis Park
Fairlawn
Penn Branch
Randle Highlands
Dupont Park
Twining
Navy Yard
Southwest Waterfront
Near Southeast
Marshall Heights
Southwest Employment Area
Fort Dupont
Capitol View
Benning Heights
Stanton Park
Kingman Park
Carver Langston
Foggy Bottom
Burrville
Mayfair
Eastland Gardens
Deanwood
Dupont Circle
Logan Circle
Georgetown Reservoir
Truxton Circle
Ivy City
Shaw
Foxhall Village
Burleith
Le Droit Park
Cardozo
Adams Morgan
Howard University
Glover Park
Foxhall Crescent
Woodland-Normanstone Terrace
Fort Lincoln
Lanier Heights
Massachusetts Avenue Heights
Woodley Park
Palisades
Columbia Heights
Pleasant Plains
Wesley Heights
Cathedral Heights
Cleveland Park
McLean Gardens
Petworth
Van Ness
Michigan Pa

In [33]:
neighborhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hillcrest,29.392196,-95.224491,Cooking Club,29.391011,-95.219880,American Restaurant
1,Shepherd Park,29.817628,-95.409891,Cottonwood,29.818285,-95.410011,Beer Garden
2,Shepherd Park,29.817628,-95.409891,Shepherd Park Draught House,29.817630,-95.409909,Beer Garden
3,Shepherd Park,29.817628,-95.409891,Starbucks,29.814732,-95.411029,Coffee Shop
4,Shepherd Park,29.817628,-95.409891,Pink's Pizza,29.817681,-95.409936,Pizza Place


In [34]:
neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Adams Morgan,76,76,76,76,76,76
American University Park,2,2,2,2,2,2
Barnaby Woods,4,4,4,4,4,4
Barry Farm,9,9,9,9,9,9
Benning Heights,13,13,13,13,13,13
Burleith,3,3,3,3,3,3
Burrville,6,6,6,6,6,6
Buzzard Point,8,8,8,8,8,8
Capitol View,5,5,5,5,5,5


In [35]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 265 uniques categories.


### Analyze Each Neighborhood In Washington DC

In [36]:
# Analyze Each Neighborhood
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Library,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Pub,Public Bathroom,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [37]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,Airport Terminal,American Restaurant,Arcade,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Cantonese Restaurant,Caribbean Restaurant,Chaat Place,Chinese Restaurant,Chiropractor,Clothing Store,Cocktail Bar,Coffee Shop,College Administrative Building,College Library,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Cuban Restaurant,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Drugstore,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Football Stadium,Forest,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Home Service,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,National Park,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Opera House,Optical Shop,Other Repair Shop,Outdoor Sculpture,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pilates Studio,Pizza Place,Planetarium,Playground,Plaza,Pool,Pop-Up Shop,Portuguese Restaurant,Pub,Public Bathroom,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Reservoir,Residential Building (Apartment / Condo),Restaurant,River,Road,Rock Club,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio,Zoo
0,Adams Morgan,0.000000,0.000000,0.013158,0.000000,0.000000,0.000000,0.000000,0.000000,0.013158,0.000000,0.000000,0.026316,0.000000,0.000000,0.013158,0.013158,0.013158,0.000000,0.065789

In [38]:
neighborhood_grouped.shape

(84, 265)

In [39]:
#Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [40]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adams Morgan,Bar,Coffee Shop,Ethiopian Restaurant,Pizza Place,Ice Cream Shop,Café,Spa,Asian Restaurant,Park,Diner
1,American University Park,Food Truck,Art Gallery,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Empanada Restaurant
2,Barnaby Woods,BBQ Joint,Field,Gym / Fitness Center,Park,Zoo,Falafel Restaurant,Ethiopian Restaurant,Event Service,Event Space,Exhibit
3,Barry Farm,Bus Stop,Spa,Basketball Court,Convenience Store,Rental Car Location,Metro Station,Bus Station,Gym,Exhibit,Falafel Restaurant
4,Benning Heights,Gay Bar,Restaurant,Fast Food Restaurant,Café,Fried Chicken Joint,Liquor Store,Sandwich Place,Pharmacy,Grocery Store,Fish & Chips Shop


In [41]:
# Cluster Neighborhoods
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0], dtype=int32)

In [42]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

washingtondc_merged = washingtondc_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
washingtondc_merged = washingtondc_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

washingtondc_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,20020,Hillcrest,Hillcrest,20020,29.392196,-95.224491,3.0,American Restaurant,Zoo,Fast Food Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Field
1,20012,Shepherd Park,Shepherd Park,20012,29.817628,-95.409891,0.0,Pizza Place,Beer Garden,BBQ Joint,Park,Burger Joint,Mexican Restaurant,Food,Business Service,Fast Food Restaurant,Beer Store
2,20019,River Terrace,River Terrace,20019,30.101878,-95.133057,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20016,Spring Valley,Spring Valley,20016,32.729640,-116.966862,0.0,Mexican Restaurant,Chinese Restaurant,Dessert Shop,Cosmetics Shop,Convenience Store,Vietnamese Restaurant,Sandwich Place,Laundromat,Lounge,Zoo
4,20007,Georgetown,Georgetown,20007,33.376834,-79.294496,0.0,Seafood Restaurant,American Restaurant,Chinese Restaurant,Food,Grocery Store,Discount Store,Pharmacy,Convenience Store,Diner,Filipino Restaurant


In [43]:
# Visualize the resulting clusters
# Get the geographical coordinates of Washington DC.
address = 'Washington DC'
geolocator = Nominatim(user_agent="usa_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(washingtondc_merged['Latitude'], washingtondc_merged['Longitude'], washingtondc_merged['Neighborhood'], washingtondc_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#       color=rainbow[cluster-1],
        fill=True,
#        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Washington DC Custers Below

In [44]:
washingtondc_merged.loc[washingtondc_merged['Cluster Labels'] == 0, washingtondc_merged.columns[[1] + list(range(5, washingtondc_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Shepherd Park,-95.409891,0.0,Pizza Place,Beer Garden,BBQ Joint,Park,Burger Joint,Mexican Restaurant,Food,Business Service,Fast Food Restaurant,Beer Store
3,Spring Valley,-116.966862,0.0,Mexican Restaurant,Chinese Restaurant,Dessert Shop,Cosmetics Shop,Convenience Store,Vietnamese Restaurant,Sandwich Place,Laundromat,Lounge,Zoo
4,Georgetown,-79.294496,0.0,Seafood Restaurant,American Restaurant,Chinese Restaurant,Food,Grocery Store,Discount Store,Pharmacy,Convenience Store,Diner,Filipino Restaurant
5,Fort McNair,-82.139603,0.0,Food Court,American Restaurant,Coffee Shop,Gym,Pizza Place,Field,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Electronics Store
6,Fort Stanton,-105.523351,0.0,National Park,Historic Site,Zoo,Farm,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Fast Food Restaurant
8,Hillbrook,-78.633948,0.0,Music Venue,Construction & Landscaping,Zoo,Farmers Market,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Field
9,Colonial Village,-78.884720,0.0,Coffee Shop,Home Service,Zoo,Farmers Market,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Fast Food Restaurant
10,Forest Hills,-97.230823,0.0,Photography Studio,Dance Studio,Department Store,Gym,Road,Zoo,Event Service,Event Space,Exhibit,Falafel Restaurant
11,Chevy Chase,-84.486135,0.0,American Restaurant,Bank,Pharmacy,Ice Cream Shop,Dry Cleaner,Insurance Office,Supermarket,Restaurant,Event Service,Event Space
12,Crestwood,-90.379390,0.0,Clothing Store,Basketball Court,Playground,Music Store,Pizza Place,Construction & Landscaping,BBQ Joint,Farm,Event Space,Exhibit


In [74]:
washingtondc_merged.loc[washingtondc_merged['Cluster Labels'] == 1, washingtondc_merged.columns[[1] + list(range(5, washingtondc_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Capitol View,-76.924900,1.0,Park,Cosmetics Shop,Café,Bed & Breakfast,Bus Station,Zoo,Farm,Event Space,Exhibit,Falafel Restaurant
88,Woodland,-111.227121,1.0,Food,Café,Bed & Breakfast,Zoo,Farmers Market,Event Space,Exhibit,Falafel Restaurant,Farm,Fast Food Restaurant


In [46]:
washingtondc_merged.loc[washingtondc_merged['Cluster Labels'] == 2, washingtondc_merged.columns[[1] + list(range(5, washingtondc_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Garfield Heights,-76.972746,2.0,Convenience Store,Museum,Wings Joint,Chinese Restaurant,Zoo,Farm,Ethiopian Restaurant,Event Service,Event Space,Exhibit
19,Fairfax Village,-76.951749,2.0,Wings Joint,Convenience Store,Bike Rental / Bike Share,Pizza Place,Shopping Mall,Zoo,Farm,Event Service,Event Space,Exhibit
21,Fort Davis Park,-76.949141,2.0,Wings Joint,Convenience Store,Bike Rental / Bike Share,Pizza Place,Shopping Mall,Zoo,Farm,Event Service,Event Space,Exhibit
23,Penn Branch,-76.953623,2.0,Food Truck,Wings Joint,Bike Rental / Bike Share,Convenience Store,Laundromat,Fish & Chips Shop,Fish Market,Flower Shop,Filipino Restaurant,Ethiopian Restaurant


In [47]:
washingtondc_merged.loc[washingtondc_merged['Cluster Labels'] == 3, washingtondc_merged.columns[[1] + list(range(5, washingtondc_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hillcrest,-95.224491,3.0,American Restaurant,Zoo,Fast Food Restaurant,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Field


In [48]:
washingtondc_merged.loc[washingtondc_merged['Cluster Labels'] == 4, washingtondc_merged.columns[[1] + list(range(5, washingtondc_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Eastland Gardens,-76.943617,4.0,Fast Food Restaurant,Snack Place,Bookstore,Event Service,Event Space,Exhibit,Falafel Restaurant,Farm,Farmers Market,Zoo
71,Michigan Park,-76.994335,4.0,Fast Food Restaurant,Zoo,Empanada Restaurant,Food,Flower Shop,Fish Market,Fish & Chips Shop,Filipino Restaurant,Field,Farmers Market


# OTTAWA

### Explore Ottawa

In [52]:
# Get the geographical coordinates of Ottawa.
latitude = 45.421
longitude = -75.69
print('The geograpical coordinate of Ottawa are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Ottawa are 45.421, -75.69.


In [53]:
# create map of Ottawa using latitude and longitude values
map_ottawa = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(ottawa_neighborhoods['Latitude'], ottawa_neighborhoods['Longitude'], ottawa_neighborhoods['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_mexico_city)
    
# Display the map
map_ottawa

In [54]:
# Get Ottawa Venues
LIMIT = 100
neighborhood_venues = getNearbyVenues(names=ottawa_neighborhoods['Neighborhood'],
                                    latitudes=ottawa_neighborhoods['Latitude'],
                                    longitudes=ottawa_neighborhoods['Longitude']
                                    )

print(neighborhood_venues.shape)

Parliament Hill
K1A 0A1
K1A 0A2
K1A 0A3
K1A 0A4
K1A 0A5
K1A 0A6
K1A 0A7
K1A 0A8
K1A 0A9
K1A 0B1
K1A 0B2
K1A 0B2
K1A 0B4
K1A 0B9
K1A 0C1
K1A 0C5
K1A 0C6
K1A 0C7
K1A 0C8
K1A 0E4
K1A 0E6
K1A 0E8
K1A 0E9
K1A 0G1
K1A 0G2
K1A 0G5
K1A 0G6
K1A 0G7
K1A 0G8
K1A 0G9
K1A 0H2
K1A 0H5
K1A 0H7
K1A 0H8
K1A 0H9
(962, 7)


In [55]:
neighborhood_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parliament Hill,45.423594,-75.700929,Northern Lights Sound and Light Show,45.423856,-75.698520,Plaza
1,Parliament Hill,45.423594,-75.700929,Riviera,45.423165,-75.696205,Restaurant
2,Parliament Hill,45.423594,-75.700929,Sheraton Ottawa Hotel,45.420935,-75.697866,Hotel
3,Parliament Hill,45.423594,-75.700929,Cafe Deluxe,45.421744,-75.696310,Café
4,Parliament Hill,45.423594,-75.700929,Bier Markt,45.421875,-75.699557,Belgian Restaurant


In [56]:
neighborhood_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
K1A 0A1,5,5,5,5,5,5
K1A 0A2,5,5,5,5,5,5
K1A 0A3,88,88,88,88,88,88
K1A 0A4,89,89,89,89,89,89
K1A 0A5,6,6,6,6,6,6
K1A 0A6,89,89,89,89,89,89
K1A 0A7,7,7,7,7,7,7
K1A 0A8,5,5,5,5,5,5
K1A 0A9,89,89,89,89,89,89


In [57]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(neighborhood_venues['Venue Category'].unique())))

There are 110 uniques categories.


### Analyze Each Neighborhood In Ottawa

In [58]:
# Analyze Each Neighborhood
# one hot encoding
neighborhood_onehot = pd.get_dummies(neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhood_onehot['Neighborhood'] = neighborhood_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhood_onehot.columns[-1]] + list(neighborhood_onehot.columns[:-1])
neighborhood_onehot = neighborhood_onehot[fixed_columns]

neighborhood_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Art Gallery,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Belgian Restaurant,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Stop,Café,Camera Store,Caribbean Restaurant,Clothing Store,Coffee Shop,College Arts Building,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Deli / Bodega,Dessert Shop,Diner,Dog Run,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gastropub,German Restaurant,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Library,Light Rail Station,Lounge,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Museum,Neighborhood,New American Restaurant,Newsstand,Noodle House,Office,Other Great Outdoors,Park,Persian Restaurant,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Pub,Record Shop,Restaurant,Rock Climbing Spot,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Vietnamese Restaurant,Waterfall,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parliament Hill,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parliament Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parliament Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parliament Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Parliament Hill,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [59]:
neighborhood_grouped = neighborhood_onehot.groupby('Neighborhood').mean().reset_index()
neighborhood_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Art Gallery,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Beer Bar,Belgian Restaurant,Bookstore,Breakfast Spot,Brewery,Bridal Shop,Burger Joint,Bus Stop,Café,Camera Store,Caribbean Restaurant,Clothing Store,Coffee Shop,College Arts Building,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Deli / Bodega,Dessert Shop,Diner,Dog Run,Food Truck,French Restaurant,Furniture / Home Store,Garden,Gastropub,German Restaurant,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,Hobby Shop,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Juice Bar,Karaoke Bar,Korean Restaurant,Library,Light Rail Station,Lounge,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Museum,New American Restaurant,Newsstand,Noodle House,Office,Other Great Outdoors,Park,Persian Restaurant,Pharmacy,Pizza Place,Plaza,Portuguese Restaurant,Pub,Record Shop,Restaurant,Rock Climbing Spot,Salad Place,Sandwich Place,Sculpture Garden,Seafood Restaurant,Skating Rink,Smoothie Shop,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Sushi Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Toy / Game Store,Vietnamese Restaurant,Waterfall,Wine Bar
0,K1A 0A1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,K1A 0A2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000
2,K1A 0A3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011364,0.000000,0.011364,0.022727,0.000000,0.0,0.000000,0.000000,0.011364,0.034091,0.000000,0.011364,0.011364,0.204545,0.000000,0.000000,0.000000,0.034091,0.000000,0.000000,0.011364,0.000000,0.000000,0.000000,0.000000,0.022727,0.011364,0.011364,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.022727,0.000000,0.000000,0.011364,0.000000,0.090909,0.000000,0.000000,0.011364,0.000000,0.022727,0.000000,0.011364,0.00

In [60]:
neighborhood_grouped.shape

(35, 110)

In [61]:
#Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [62]:
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = neighborhood_grouped['Neighborhood']

for ind in np.arange(neighborhood_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhood_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,K1A 0A1,Brewery,Sculpture Garden,Historic Site,Garden,Café,Gift Shop,Deli / Bodega,Dessert Shop,Diner,Dog Run
1,K1A 0A2,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
2,K1A 0A3,Coffee Shop,Hotel,Restaurant,Café,Sandwich Place,Concert Hall,Plaza,Museum,Bookstore,Lounge
3,K1A 0A4,Coffee Shop,Hotel,Restaurant,Concert Hall,Sandwich Place,Plaza,Café,Clothing Store,Bookstore,Japanese Restaurant
4,K1A 0A5,Waterfall,Dog Run,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Food Truck
5,K1A 0A6,Coffee Shop,Hotel,Restaurant,Concert Hall,Sandwich Place,Plaza,Café,Clothing Store,Bookstore,Japanese Restaurant
6,K1A 0A7,Pub,Hotel,Restaurant,Rock Climbing Spot,Library,Park,Memorial Site,Garden,Cosmetics Shop,Deli / Bodega
7,K1A 0A8,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
8,K1A 0A9,Coffee Shop,Hotel,Restaurant,Concert Hall,Sandwich Place,Plaza,Café,Clothing Store,Bookstore,Japanese Restaurant
9,K1A 0B1,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck


In [63]:
# Cluster Neighborhoods
# Run k-means to cluster the neighborhood into 5 clusters.

# set number of clusters
kclusters = 5

neighborhood_grouped_clustering = neighborhood_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhood_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 1, 1, 0, 1, 3, 0, 1, 0], dtype=int32)

In [64]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

ottawa_merged = ottawa_neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
ottawa_merged = ottawa_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

ottawa_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Location,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,K1A,West Block,Parliament Hill,K1A West Block,45.423594,-75.700929,1,Coffee Shop,Hotel,Restaurant,Sandwich Place,Café,Plaza,Food Truck,Gym,Museum,Concert Hall
1,K1A 0A1,K1A 0A1,K1A 0A1,K1A 0A1,45.445473,-75.689448,2,Brewery,Sculpture Garden,Historic Site,Garden,Café,Gift Shop,Deli / Bodega,Dessert Shop,Diner,Dog Run
2,K1A 0A2,K1A 0A2,K1A 0A2,K1A 0A2,45.444840,-75.691540,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
3,K1A 0A3,K1A 0A3,K1A 0A3,K1A 0A3,45.423382,-75.697892,1,Coffee Shop,Hotel,Restaurant,Café,Sandwich Place,Concert Hall,Plaza,Museum,Bookstore,Lounge
4,K1A 0A4,K1A 0A4,K1A 0A4,K1A 0A4,45.423787,-75.697460,1,Coffee Shop,Hotel,Restaurant,Concert Hall,Sandwich Place,Plaza,Café,Clothing Store,Bookstore,Japanese Restaurant


In [65]:
# Visualize the resulting clusters

# Get the latitude and longitude of Ottawa.


# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(ottawa_merged['Latitude'], ottawa_merged['Longitude'], ottawa_merged['Neighborhood'], ottawa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
#       color=rainbow[cluster-1],
        fill=True,
#        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Ottawa Clusters Below

In [66]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 0, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]


,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,K1A 0A2,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
5,K1A 0A5,-75.69156,0,Waterfall,Dog Run,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Food Truck
8,K1A 0A8,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
10,K1A 0B1,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
13,K1A 0B4,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
14,K1A 0B9,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
15,K1A 0C1,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
16,K1A 0C5,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
18,K1A 0C7,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck
19,K1A 0C8,-75.69154,0,Waterfall,Park,Historic Site,Bar,Café,Gift Shop,Dessert Shop,Diner,Dog Run,Food Truck


In [67]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 1, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,West Block,-75.700929,1,Coffee Shop,Hotel,Restaurant,Sandwich Place,Café,Plaza,Food Truck,Gym,Museum,Concert Hall
3,K1A 0A3,-75.697892,1,Coffee Shop,Hotel,Restaurant,Café,Sandwich Place,Concert Hall,Plaza,Museum,Bookstore,Lounge
4,K1A 0A4,-75.697460,1,Coffee Shop,Hotel,Restaurant,Concert Hall,Sandwich Place,Plaza,Café,Clothing Store,Bookstore,Japanese Restaurant
6,K1A 0A6,-75.697460,1,Coffee Shop,Hotel,Restaurant,Concert Hall,Sandwich Place,Plaza,Café,Clothing Store,Bookstore,Japanese Restaurant
9,K1A 0A9,-75.697460,1,Coffee Shop,Hotel,Restaurant,Concert Hall,Sandwich Place,Plaza,Café,Clothing Store,Bookstore,Japanese Restaurant
21,K1A 0E6,-75.701051,1,Coffee Shop,Hotel,Sushi Restaurant,Greek Restaurant,Food Truck,Café,Middle Eastern Restaurant,Tapas Restaurant,Gym,Italian Restaurant
22,K1A 0E8,-75.705606,1,Italian Restaurant,Coffee Shop,Pub,Wine Bar,Spa,Korean Restaurant,Mediterranean Restaurant,Greek Restaurant,Molecular Gastronomy Restaurant,Diner
26,K1A 0G5,-75.696918,1,Coffee Shop,Café,Restaurant,Salad Place,Italian Restaurant,Japanese Restaurant,Sandwich Place,Smoothie Shop,Greek Restaurant,Gym
28,K1A 0G7,-75.700194,1,Coffee Shop,Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Greek Restaurant,Café,Food Truck,Sandwich Place,Hotel,Japanese Restaurant
29,K1A 0G8,-75.698643,1,Dessert Shop,Coffee Shop,Yoga Studio,Spa,Jazz Club,Gift Shop,German Restaurant,Furniture / Home Store,Museum,Diner


In [68]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 2, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,K1A 0A1,-75.689448,2,Brewery,Sculpture Garden,Historic Site,Garden,Café,Gift Shop,Deli / Bodega,Dessert Shop,Diner,Dog Run


In [69]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 3, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,K1A 0A7,-75.71000,3,Pub,Hotel,Restaurant,Rock Climbing Spot,Library,Park,Memorial Site,Garden,Cosmetics Shop,Deli / Bodega
11,K1A 0B2,-75.69154,3,Park,Bar,Library,Memorial Site,Rock Climbing Spot,Restaurant,Café,Historic Site,Pub,Waterfall
12,K1A 0B2,-75.71000,3,Park,Bar,Library,Memorial Site,Rock Climbing Spot,Restaurant,Café,Historic Site,Pub,Waterfall


In [70]:
ottawa_merged.loc[ottawa_merged['Cluster Labels'] == 4, ottawa_merged.columns[[1] + list(range(5, ottawa_merged.shape[1]))]]

,Borough,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,K1A 0C6,-75.711899,4,Wine Bar,Sports Club,Indian Restaurant,Light Rail Station,Mediterranean Restaurant,Harbor / Marina,Gym / Fitness Center,Greek Restaurant,Park,Diner
